# Creating Ontology of 311 Park Names -> official park names

This notebook creates a match dictionary between park names in 311 data and in NYC Parks official dataset. 
Using this dictionary we can match each park-related call to specific park or park District

Here I am using [fuzzywuzzy module](https://github.com/seatgeek/fuzzywuzzy)

In [152]:
__author__ = "Philipp Kats"
__date__ = "2015_10_15"

import pandas as pd
import geopandas as gp
import numpy as np
import os
from fuzzywuzzy import process  ## fuzzy string match in python

%pylab inline
PARQA = os.getenv('PARQA') #basic path

Populating the interactive namespace from numpy and matplotlib


need to add: recreation centers,
    todd lot, and remove PS

## Getting 311 Data

In [153]:
calls = pd.read_csv(PARQA + '/data/311DPR.csv',encoding='utf8', na_values='Unspecified')
calls= calls[pd.notnull(calls['Park Facility Name'])]

In [154]:
callParks = calls[['Park Facility Name','Borough']].drop_duplicates()
calls[pd.notnull(calls['Park Facility Name'])]
callParks['cleanName'] = callParks['Park Facility Name'].str.lower()
print '311_call_names total:', len(callParks['Park Facility Name'])

311_call_names total: 1666


In [155]:
callParks.head()

,Park Facility Name,Borough,cleanName
1,Geo Soilan Park - Battery Park City,MANHATTAN,geo soilan park - battery park city
2,Brookville Park,QUEENS,brookville park
5,Highland Park,BROOKLYN,highland park
10,Prospect Park - East Parade Grounds,BROOKLYN,prospect park - east parade grounds
11,Central Park - East 96th Street Playground,MANHATTAN,central park - east 96th street playground


In [156]:
borrowCode = pd.DataFrame({'Borough':['BROOKLYN',
                                      'QUEENS',
                                      'MANHATTAN',
                                      'BRONX',
                                      'STATEN ISLAND',
                                      'Unspecified'],
                           'code':['B',
                                   'Q',
                                   'M',
                                   'X',
                                   'R',
                                   '?']})

callParks = callParks.merge(borrowCode, how='left', on='Borough')
callParks.head(3)

,Park Facility Name,Borough,cleanName,code
0,Geo Soilan Park - Battery Park City,MANHATTAN,geo soilan park - battery park city,M
1,Brookville Park,QUEENS,brookville park,Q
2,Highland Park,BROOKLYN,highland park,B


## Getting Park names

In [157]:
## using geojson produced of OPEN DATA 
parkNames = gp.read_file(PARQA + 'data/parks_computed.geojson')[['SIGNNAME','BOROUGH']].drop_duplicates()
parkNames['SIGNNAME'] = parkNames['SIGNNAME'].str.lower()
len(parkNames['SIGNNAME'].drop_duplicates())

1787

In [158]:
#create a dictionary of parks for each borough 
#that will add accuracy and boost spead of search ( I hope)

ofParks = {k:[unicode(x) for x in list(v)] for k,v in parkNames.groupby("BOROUGH")["SIGNNAME"]}

tmp = []
for v in ofParks.values():
    tmp.extend(v)
ofParks['?'] = tmp   #['???'] ### to mark parks with unspecified Borough

print ofParks.keys()

[u'B', u'M', u'Q', u'R', u'X', '?']


In [159]:
# ofParks['B']

## I. Direct matching

In [160]:
cParksMatched = callParks.merge(parkNames, how='left', left_on = 'cleanName', right_on='SIGNNAME', copy=1)
cParksMatched['Type'] = np.nan

In [161]:
len(cParksMatched[pd.notnull(cParksMatched.SIGNNAME)])

714

In [162]:
cParksMatched.ix[(pd.notnull(cParksMatched.SIGNNAME)&pd.isnull(cParksMatched.Type)), 'Type'] = 'park_direct'

In [163]:
cParksMatched.head()

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type
0,Geo Soilan Park - Battery Park City,MANHATTAN,geo soilan park - battery park city,M,NaN,NaN,NaN
1,Brookville Park,QUEENS,brookville park,Q,brookville park,Q,park_direct
2,Highland Park,BROOKLYN,highland park,B,highland park,B,park_direct
3,Highland Park,BROOKLYN,highland park,B,highland park,Q,park_direct
4,Prospect Park - East Parade Grounds,BROOKLYN,prospect park - east parade grounds,B,NaN,NaN,NaN


## II. Empiric decoding

In [164]:
#some manual improvisation, based on emirical experience

def empiric(x):
    '''returns clean keyName if name contains 
    specific words (central park, for example)'''

    general = {'flushing meadows':'flushing meadows and corona park',
               'central park':'central park', 
               'prospect park':'prospect park',
               'greenbelt': 'greenbelt native plant center',
               'red hook park':'red hook recreation area',
               'crotona park':'crotona park',
               'marine park': 'marine park', 
               'van cortlandt':'van cortlandt park',
               'seravalli': 'corporal john a. seravalli playground',
               'inwood hill': 'inwood hill park',
               'forest park':'forest park',
               'pelham bay park':'pelham bay park',
               'kissena park': 'kissena park',
               'asser levy': 'asser levy park',
               'highland park':'highland park',
               'bronx park':'bronx park',
               '100 percent':'100% Playground',
               'seaside playground':'Seaside Playground',
               'dolen':'Owen F. Dolen Park',
               'loring':'Vito Locascio Field'
                }
    
    
    for k in general.keys():
        if k in x.cleanName:
            return general[k]
    return np.nan
    

In [165]:
cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME), 'SIGNNAME'] = cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME)].apply(empiric,1)

In [166]:
cParksMatched.ix[(pd.notnull(cParksMatched.SIGNNAME)&pd.isnull(cParksMatched.Type)), 'Type'] = 'empiric'

In [167]:
cParksMatched.head(10)

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type
0,Geo Soilan Park - Battery Park City,MANHATTAN,geo soilan park - battery park city,M,NaN,NaN,NaN
1,Brookville Park,QUEENS,brookville park,Q,brookville park,Q,park_direct
2,Highland Park,BROOKLYN,highland park,B,highland park,B,park_direct
3,Highland Park,BROOKLYN,highland park,B,highland park,Q,park_direct
4,Prospect Park - East Parade Grounds,BROOKLYN,prospect park - east parade grounds,B,prospect park,NaN,empiric
5,Central Park - East 96th Street Playground,MANHATTAN,central park - east 96th street playground,M,central park,NaN,empiric
6,Washington Hall Park,BROOKLYN,washington hall park,B,washington hall park,B,park_direct
7,Callahan-Kelly Playground,BROOKLYN,callahan-kelly playground,B,callahan-kelly playground,B,park_direct
8,William H Seward Park,MANHATTAN,william h seward park,M,NaN,NaN,NaN
9,Col David Marcus Playground,BROOKLYN,col david marcus playground,B,NaN,NaN,NaN


In [168]:
print 'Recognized:'
print len(cParksMatched[pd.notnull(cParksMatched.SIGNNAME)]), '/', len(cParksMatched)


Recognized:
923 / 1678


## III. Playgrounds

In [169]:
playgrounds = [x.decode('utf8') for x in pd.read_csv(PARQA + 'data/DPR_property/playgrounds.csv')['Name'].tolist()]

In [170]:
# cParksMatched[pd.isnull(cParksMatched.SIGNNAME)][cParksMatched.cleanName.str.contains('play')]

In [171]:
cParksMatched['match'] = np.nan
cParksMatched['ratio'] = np.nan

In [172]:
cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME) & cParksMatched.cleanName.str.contains('play'), 'Type'] = 'pgs'
cParksMatched.ix[cParksMatched.Type=='pgs', 'match'] = \
cParksMatched.ix[cParksMatched.Type=='pgs', 'cleanName'].apply(lambda x: process.extractOne(x, playgrounds))

In [173]:
cParksMatched.ix[cParksMatched.Type=='pgs', 'SIGNNAME'] = \
cParksMatched.ix[cParksMatched.Type=='pgs', 'match'].apply(lambda x: x[0])

cParksMatched.ix[cParksMatched.Type=='pgs', 'ratio'] = \
cParksMatched.ix[cParksMatched.Type=='pgs', 'match'].apply(lambda x: x[1])

In [174]:
cParksMatched[cParksMatched.Type=='pgs']

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type,match,ratio
9,Col David Marcus Playground,BROOKLYN,col david marcus playground,B,Col David Marcus Mem Playground,NaN,pgs,"(Col David Marcus Mem Playground, 95)",95
13,Mccarren Park - Vincent V Abate Playground,BROOKLYN,mccarren park - vincent v abate playground,B,Vincent V Abate Playground,NaN,pgs,"(Vincent V Abate Playground, 90)",90
24,Harold L Ickes Playground,BROOKLYN,harold l ickes playground,B,Harold Ickes Playground,NaN,pgs,"(Harold Ickes Playground, 96)",96
27,Hoover Manton Playground,QUEENS,hoover manton playground,Q,Manton Playground,NaN,pgs,"(Manton Playground, 95)",95
33,Emerald Playground - PS 200,QUEENS,emerald playground - ps 200,Q,Playground,NaN,pgs,"(Playground, 90)",90
36,Doctor Gertrude B Kelly Playground,MANHATTAN,doctor gertrude b kelly playground,M,Dr Gertrude B Kelly Playground,NaN,pgs,"(Dr Gertrude B Kelly Playground, 94)",94
46,May Matthews Playground,MANHATTAN,may matthews playground,M,May Matthews Playground,NaN,pgs,"(May Matthews Playground, 100)",100
47,Cuyler Gore Playground,BROOKLYN,cuyler gore playground,B,Cuyler Gore,NaN,pgs,"(Cuyler Gore, 90)",90
52,Terrapin Playground - JHS 51,BROOKLYN,terrapin playground - jhs 51,B,Terrapin Playground,NaN,pgs,"(Terrapin Playground, 95)",95
53,Jamaica Playground - PS 40,QUEENS,jamaica playground - ps 40,Q,Jamaica Playground (ps 40),NaN,pgs,"(Jamaica Playground (ps 40), 98)",98


In [175]:
print 'Recognized:'
print len(cParksMatched[pd.notnull(cParksMatched.SIGNNAME)]), '/', len(cParksMatched)


Recognized:
1302 / 1678


## IV. Pools

In [176]:
pools = [x.decode('utf8').lower() for x in pd.read_csv(PARQA + 'data/DPR_property/pools.csv')['Name'].tolist()]

In [177]:
cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME) & cParksMatched.cleanName.str.contains('pool'), 'Type'] = 'pool'
# cParksMatched[cParksMatched.Type=='pool']


In [178]:
cParksMatched.ix[cParksMatched.Type=='pool', 'match'] = \
cParksMatched.ix[cParksMatched.Type=='pool', 'cleanName'].apply(lambda x: process.extractOne(x, pools))

In [179]:
cParksMatched.ix[cParksMatched.Type=='pool', 'SIGNNAME'] = \
cParksMatched.ix[cParksMatched.Type=='pool', 'match'].apply(lambda x: x[0])

cParksMatched.ix[cParksMatched.Type=='pool', 'ratio'] = \
cParksMatched.ix[cParksMatched.Type=='pool', 'match'].apply(lambda x: x[1])

In [180]:
cParksMatched[cParksMatched.Type=='pool']

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type,match,ratio
140,Pool - Metropolitan Avenue,BROOKLYN,pool - metropolitan avenue,B,claremont pool,NaN,pool,"(claremont pool, 85)",85
156,Pool - St. John's Recreation Center,BROOKLYN,pool - st. john's recreation center,B,recreation center 54 pool,NaN,pool,"(recreation center 54 pool, 89)",89
179,Pool - Brownsville Recreation Center,BROOKLYN,pool - brownsville recreation center,B,recreation center 54 pool,NaN,pool,"(recreation center 54 pool, 89)",89
218,Recreation Center - Metropolitan Pool and Fitn...,BROOKLYN,recreation center - metropolitan pool and fitn...,B,metropolitan pool,NaN,pool,"(metropolitan pool, 90)",90
309,Pool - Hansborough,MANHATTAN,pool - hansborough,M,hansborough pool,NaN,pool,"(hansborough pool, 95)",95
352,Pool - JHS 57 and HS 26,BROOKLYN,pool - jhs 57 and hs 26,B,jhs 57/hs 26 pool,NaN,pool,"(jhs 57/hs 26 pool, 95)",95
416,John Jay Park and Pool,MANHATTAN,john jay park and pool,M,claremont pool,NaN,pool,"(claremont pool, 85)",85
482,Williamsbridge Oval - P G Mini Pool and Courts,BRONX,williamsbridge oval - p g mini pool and courts,X,claremont pool,NaN,pool,"(claremont pool, 85)",85
602,Pool - Mullaly,BRONX,pool - mullaly,X,mullaly pool,NaN,pool,"(mullaly pool, 95)",95
614,Pool - Hamilton Fish,MANHATTAN,pool - hamilton fish,M,hamilton fish pool,NaN,pool,"(hamilton fish pool, 95)",95


In [181]:
print 'Recognized:'
print len(cParksMatched[pd.notnull(cParksMatched.SIGNNAME)]), '/', len(cParksMatched)

Recognized:
1360 / 1678


## V. Beaches

In [182]:
beaches = [x.decode('utf8').lower() for x in pd.read_csv(PARQA + 'data/DPR_property/beaches.csv')['Name'].tolist()]

In [183]:
cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME) & cParksMatched.cleanName.str.contains('beach'), 'Type'] = 'beach'
# cParksMatched[pd.isnull(cParksMatched.SIGNNAME)]

In [184]:
cParksMatched.ix[cParksMatched.Type=='beach', 'match'] = \
cParksMatched.ix[cParksMatched.Type=='beach', 'cleanName'].apply(lambda x: process.extractOne(x, beaches))

In [185]:
cParksMatched.ix[cParksMatched.Type=='beach', 'SIGNNAME'] = \
cParksMatched.ix[cParksMatched.Type=='beach', 'match'].apply(lambda x: x[0])

cParksMatched.ix[cParksMatched.Type=='beach', 'ratio'] = \
cParksMatched.ix[cParksMatched.Type=='beach', 'match'].apply(lambda x: x[1])

In [186]:
cParksMatched[cParksMatched.Type=='beach']

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type,match,ratio
117,Beach - Midland,STATEN ISLAND,beach - midland,R,orchard beach and promenade,NaN,beach,"(orchard beach and promenade, 85)",85
196,Beach - Brighton,BROOKLYN,beach - brighton,B,brighton beach,NaN,beach,"(brighton beach, 95)",95
223,Beach - Orchard,BRONX,beach - orchard,X,orchard beach and promenade,NaN,beach,"(orchard beach and promenade, 85)",85
361,Beach - Manhattan,BROOKLYN,beach - manhattan,B,manhattan beach,NaN,beach,"(manhattan beach, 95)",95
648,Beach - Wolfe's Pond,STATEN ISLAND,beach - wolfe's pond,R,wolfe's pond beach,NaN,beach,"(wolfe's pond beach, 95)",95
685,Beach - South,STATEN ISLAND,beach - south,R,south beach,NaN,beach,"(south beach, 95)",95
730,Crescent Beach,STATEN ISLAND,crescent beach,R,orchard beach and promenade,NaN,beach,"(orchard beach and promenade, 85)",85
785,Coney Island Beach and Boardwalk,BROOKLYN,coney island beach and boardwalk,B,coney island and coney island boardwalk,NaN,beach,"(coney island and coney island boardwalk, 95)",95
1029,Golf Course - Dyker Beach,BROOKLYN,golf course - dyker beach,B,manhattan beach,NaN,beach,"(manhattan beach, 85)",85
1217,Orchard Beach Park - Nature Center,BRONX,orchard beach park - nature center,X,manhattan beach,NaN,beach,"(manhattan beach, 85)",85


In [187]:
print 'Recognized:'
print len(cParksMatched[pd.notnull(cParksMatched.SIGNNAME)]), '/', len(cParksMatched)

Recognized:
1373 / 1678


## VI. School Playgrounds

In [188]:
schools = pd.read_csv(PARQA + 'data/DPR_property/school_playgrounds.csv')['PSID'].tolist()

In [189]:
cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME) & cParksMatched.cleanName.str.contains('ps '), 'Type'] = 'school'
# cParksMatched[pd.isnull(cParksMatched.SIGNNAME) & cParksMatched.cleanName.str.contains('ps')]
# cParksMatched[cParksMatched.cleanName.str.contains('ps')]

In [190]:
cParksMatched.ix[cParksMatched.Type=='school', 'match'] = \
cParksMatched.ix[cParksMatched.Type=='school', 'cleanName'].apply(lambda x: process.extractOne(x, schools))

In [191]:
cParksMatched.ix[cParksMatched.Type=='school', 'SIGNNAME'] = \
cParksMatched.ix[cParksMatched.Type=='school', 'match'].apply(lambda x: x[0])

cParksMatched.ix[cParksMatched.Type=='school', 'ratio'] = \
cParksMatched.ix[cParksMatched.Type=='school', 'match'].apply(lambda x: x[1])

In [192]:
cParksMatched[cParksMatched.Type=='school']

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type,match,ratio
100,Evergreen Pk - PS 68,QUEENS,evergreen pk - ps 68,Q,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85
185,Oakland Gardens - PS 203,QUEENS,oakland gardens - ps 203,Q,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85
410,Athens Square - PS 17,QUEENS,athens square - ps 17,Q,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85
464,Alfred Smith - PS 1,MANHATTAN,alfred smith - ps 1,M,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85
983,Israel Putnam - PS 44,BROOKLYN,israel putnam - ps 44,B,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85
1101,Gunn Park - PS 18,QUEENS,gunn park - ps 18,Q,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85
1336,Wayanda Park - PS 34,QUEENS,wayanda park - ps 34,Q,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85
1480,PS 4 Paradise Garden,BROOKLYN,ps 4 paradise garden,B,Ps/is 187,NaN,school,"(Ps/is 187, 85)",85


## VII Golf Courses

In [193]:
golfs = [unicode(x).lower() for x in pd.read_csv(PARQA + 'data/DPR_property/DPR_GolfCourses_001.csv')['NAME'].tolist()]

In [194]:
cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME) & cParksMatched.cleanName.str.contains('golf '), 'Type'] = 'golf'

In [196]:
cParksMatched.ix[cParksMatched.Type=='golf', 'match'] = \
cParksMatched.ix[cParksMatched.Type=='golf', 'cleanName'].apply(lambda x: process.extractOne(x, golfs))

In [199]:
cParksMatched.ix[cParksMatched.Type=='golf', 'SIGNNAME'] = \
cParksMatched.ix[cParksMatched.Type=='golf', 'match'].apply(lambda x: x[0])

cParksMatched.ix[cParksMatched.Type=='golf', 'ratio'] = \
cParksMatched.ix[cParksMatched.Type=='golf', 'match'].apply(lambda x: x[1])

In [200]:
cParksMatched[cParksMatched.Type=='golf']

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type,match,ratio
148,Golf Course - South Shore,STATEN ISLAND,golf course - south shore,R,south shore golf course,NaN,golf,"(south shore golf course, 95)",95
167,Clearview Park and Golf Course,QUEENS,clearview park and golf course,Q,clearview golf course,NaN,golf,"(clearview golf course, 95)",95
310,Golf Course - Kissena,QUEENS,golf course - kissena,Q,kissena park golf course,NaN,golf,"(kissena park golf course, 95)",95
467,Golf Course - Clearview Park,QUEENS,golf course - clearview park,Q,clearview golf course,NaN,golf,"(clearview golf course, 95)",95
495,Golf Course - Silver Lake,STATEN ISLAND,golf course - silver lake,R,silver lake golf course,NaN,golf,"(silver lake golf course, 95)",95
497,Golf Course - Douglaston,QUEENS,golf course - douglaston,Q,douglaston golf course,NaN,golf,"(douglaston golf course, 95)",95
666,Golf Course - Mosholu,BRONX,golf course - mosholu,X,mosholu golf course,NaN,golf,"(mosholu golf course, 95)",95
711,Golf Course - Golden Bear Driving Range,QUEENS,golf course - golden bear driving range,Q,clearview golf course,NaN,golf,"(clearview golf course, 85)",85
815,South Shore Golf Course,STATEN ISLAND,south shore golf course,R,south shore golf course,NaN,golf,"(south shore golf course, 100)",100
1061,Golf Course - Randalls Island Driving Range,MANHATTAN,golf course - randalls island driving range,M,clearview golf course,NaN,golf,"(clearview golf course, 85)",85


In [201]:
print 'Recognized:'
print len(cParksMatched[pd.notnull(cParksMatched.SIGNNAME)]), '/', len(cParksMatched)

Recognized:
1394 / 1678


## VI. Park Names

In [202]:
parkNames = gp.read_file(PARQA + 'data/parks_computed.geojson')[['SIGNNAME','BOROUGH']].drop_duplicates()
parkNames['SIGNNAME'] = parkNames['SIGNNAME'].str.lower()

In [203]:
ofParks = [unicode(x) for x in parkNames['SIGNNAME'].tolist()]

# ofParks = {k:[unicode(x) for x in list(v)] for k,v in parkNames.groupby("BOROUGH")["SIGNNAME"]}

# tmp = []
# for v in ofParks.values():
#     tmp.extend(v)
# ofParks['?'] = tmp   #['???'] ### to mark parks with unspecified Borough

In [204]:
cParksMatched.ix[pd.isnull(cParksMatched.SIGNNAME), 'Type'] = 'other'

In [205]:
### ATTENTION, TAKES ~5 MINUTES

cParksMatched.ix[cParksMatched.Type=='other', 'match'] = \
cParksMatched[cParksMatched.Type=='other'].apply(lambda x: process.extractOne(x.cleanName, ofParks), 1)

In [206]:
cParksMatched.ix[cParksMatched.Type=='other', 'SIGNNAME'] = \
cParksMatched.ix[cParksMatched.Type=='other', 'match'].apply(lambda x: x[0])

cParksMatched.ix[cParksMatched.Type=='other', 'ratio'] = \
cParksMatched.ix[cParksMatched.Type=='other', 'match'].apply(lambda x: x[1])

In [207]:
cParksMatched[cParksMatched.Type=='other']

,Park Facility Name,Borough,cleanName,code,SIGNNAME,BOROUGH,Type,match,ratio
0,Geo Soilan Park - Battery Park City,MANHATTAN,geo soilan park - battery park city,M,battery park city,NaN,other,"(battery park city, 90)",90
8,William H Seward Park,MANHATTAN,william h seward park,M,park,NaN,other,"(park, 90)",90
18,St Catherine's Park,MANHATTAN,st catherine's park,M,st. catherine's park,NaN,other,"(st. catherine's park, 97)",97
35,Louis J Valentino Jr Park and Pier,BROOKLYN,louis j valentino jr park and pier,B,queensbridge park,NaN,other,"(queensbridge park, 85)",85
43,Hells Kitchen Park,MANHATTAN,hells kitchen park,M,hell's kitchen park,NaN,other,"(hell's kitchen park, 97)",97
45,Brooklyn Civic Center Parks,BROOKLYN,brooklyn civic center parks,B,park,NaN,other,"(park, 90)",90
51,J J Byrne Memorial Park,BROOKLYN,j j byrne memorial park,B,park,NaN,other,"(park, 90)",90
57,Sherman Creek Park,MANHATTAN,sherman creek park,M,sherman creek,NaN,other,"(sherman creek, 95)",95
58,Kolbert Park,BROOKLYN,kolbert park,B,park,NaN,other,"(park, 90)",90
77,Alice Kornegay,MANHATTAN,alice kornegay,M,alice kornegay triangle,NaN,other,"(alice kornegay triangle, 90)",90


In [208]:
cParksMatched.to_csv(PARQA + 'data/311/Ontology2_unverified.csv',encoding='utf8')